# Toxicity Type Detection

Author: Douglas Trajano ([LinkedIn](https://www.linkedin.com/in/douglas-trajano/) | [GitHub](https://github.com/DougTrajano))

In this notebook, we will train a model to detect the toxicity labels of the comments in the OLID-BR dataset.

We will use the pre-trained BERT model from the [Hugging Face Transformers](https://huggingface.co/transformers/) library.

[neuralmind/bert-base-portuguese-cased · Hugging Face](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

## Index

The notebook will be divided into seperate sections to provide a organized walk through for the training process.

1. [Setup environment](#setup-environment)
2. [Download dataset](#download-dataset)
3. [Data preprocessing](#data-preprocessing)
4. [Prepare training session](#prepare-training-session)
5. [Hyperparameter tuning](#hyperparameter-tuning)
6. [Training with best hyperparameters](#training-with-best-hyperparameters)
7. [Evaluation](#evaluation)
8. [Register best model](#register-best-model)

## Setup environment

In this section, we will prepare the environment to run the notebook.

In [1]:
import sys
from pathlib import Path

if str(Path(".").absolute().parent) not in sys.path:
    sys.path.append(str(Path(".").absolute().parent))

In [2]:
from dotenv import load_dotenv

# Initialize the env vars
load_dotenv("../.env")

True

In [4]:
import logging
import numpy as np
import pandas as pd
import mlflow
from torch.cuda import is_available
from sklearn.metrics import classification_report
from simpletransformers.classification import (
    MultiLabelClassificationModel,
    MultiLabelClassificationArgs
)

from src.utils import (
    compute_pos_weight,
    download_dataset,
    flatten
)

logging.basicConfig(level=logging.INFO)

_logger = logging.getLogger("transformers")
_logger.setLevel(logging.WARNING)

mlflow.set_experiment("toxicity-type-detection")

mlflow.start_run()

mlflow.set_tag("model", "BERT")
mlflow.set_tag("dataset", "OLID-BR")
mlflow.set_tag("framework", "simpletransformers")

## Download dataset

In this section, we will download the OLID-BR dataset from Kaggle.

In [5]:
dataset = download_dataset(["train.csv", "test.csv"])

train_df = dataset["train.csv"]
test_df = dataset["test.csv"]

del dataset

mlflow.log_param("train_shape", train_df.shape)

print(f"Train shape: {train_df.shape}")
train_df.head()

Train shape: (4765, 17)


,id,text,is_offensive,is_targeted,targeted_type,toxic_spans,health,ideology,insult,lgbtqphobia,other_lifestyle,physical_aspects,profanity_obscene,racism,religious_intolerance,sexism,xenophobia
0,430b13705cf34e13b74bc999425187c3,USER USER é muito bom. USER ^^ E claro a equip...,NOT,UNT,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False
1,c779826dc43f460cb18e8429ca443477,Pior do que adolescentezinhas de merda...são p...,OFF,UNT,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False,True,False,False,False,True,False,False,True,False
2,e64148caa4474fc79298e01d0dda8f5e,USER Toma no cu é vitamina como tu e tua prima.,OFF,TIN,GRP,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",False,False,True,False,False,False,True,False,False,False,False
3,cc66b54eeec24607a67e2259134a1cdd,"Muito bom, pena a circunstâncias serem ruins, ...",OFF,UNT,NaN,"[119, 120, 121, 122, 123, 124, 125, 126, 127, ...",False,False,True,False,False,False,False,False,False,False,False
4,a3d7839456ae4258a70298fcf637952e,"Podia ter beijo também, pra ver se o homofóbic...",OFF,UNT,NaN,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",False,False,True,False,False,False,False,False,False,False,False


In [6]:
mlflow.log_param("test_shape", test_df.shape)

print(f"Test shape: {test_df.shape}")
test_df.head()

Test shape: (1589, 17)


,id,text,is_offensive,is_targeted,targeted_type,toxic_spans,health,ideology,insult,lgbtqphobia,other_lifestyle,physical_aspects,profanity_obscene,racism,religious_intolerance,sexism,xenophobia
0,da19df36730945f08df3d09efa354876,USER Adorei o comercial também Jesus. Só achei...,OFF,UNT,NaN,"[52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6...",False,False,True,False,False,False,True,False,False,False,False
1,80f1a8c981864887b13963fed1261acc,Cara isso foi muito babaca geral USER conhece ...,OFF,TIN,GRP,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False,True,False,False,False,False,False,False,False,False
2,2f67025f913e4a6292e3d000d9e2b5a8,"Se vc for porco, folgado e relaxado, você não ...",OFF,UNT,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False,True,False,False,False,False,False,False,False,False
3,738ccd4476784f47af3a5a6cfdda4695,Se fosse um sniper ia ser louco,OFF,UNT,NaN,"[26, 27, 28, 29, 30]",False,False,True,False,False,True,False,False,False,False,False
4,e0064da693bd4c9e90ce8e6db8bd3bbb,USER é o meu saco USER USER USER,OFF,UNT,NaN,"[13, 14, 15, 16]",False,False,True,False,False,False,True,False,False,False,False


## Data preprocessing

In this section, we will preprocess the dataset to be used in the training process.

We will filter only the comments with the toxicity labels and also remove the unnecessary columns.

In [7]:
def preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess the dataset.

    Args:
    - df: The dataset to be preprocessed.

    Returns:
    - The preprocessed dataset.
    """
    import warnings

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        # Filter only offensive comments
        df = df[df["is_offensive"] == "OFF"]

        # Remove religious_intolerance that has only one  sample
        if "religious_intolerance" in df.columns:
            df.drop("religious_intolerance", axis=1, inplace=True)

        # Filter only offensive comments with at least one toxicity label
        df = df.loc[df.select_dtypes("bool").sum(axis=1).ge(1)]

        return df.reset_index(drop=True)

toxicity_labels = [
    "health",
    "ideology",
    "insult",
    "lgbtqphobia",
    "other_lifestyle",
    "physical_aspects",
    "profanity_obscene",
    "racism",
    "sexism",
    "xenophobia"
]

train_df = preprocessing(train_df)
test_df = preprocessing(test_df)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

Train shape: (4272, 16)
Test shape: (1438, 16)


In [8]:
X_train = train_df["text"].values
y_train = train_df[toxicity_labels].astype(int).values

X_test = test_df["text"].values
y_test = test_df[toxicity_labels].astype(int).values

## Prepare training session

In this section, we will prepare the training session, defining the training parameters and computing the class weights.

In [9]:
params = {
    "max_seq_length": len(max(X_train, key=len)),
    "model_name": "neuralmind/bert-base-portuguese-cased",
    "model_type": "bert",
    "num_train_epochs": 30,
    "num_train_epochs_per_child": 3,
    "seed": 1993,
    "use_cuda": is_available()
}

params

{'max_seq_length': 1084,
 'model_name': 'neuralmind/bert-base-portuguese-cased',
 'model_type': 'bert',
 'num_train_epochs': 30,
 'num_train_epochs_per_child': 3,
 'seed': 1993,
 'use_cuda': True}

In the next cell, we will compute the class weights to be used in the training process.

In [10]:
pos_weights = compute_pos_weight(y_train)

### Create Estimator

In [11]:
import os
import shutil
from typing import Any, Dict, List, Union
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report

class ToxicityTypeDetection(BaseEstimator):
    def __init__(self,
                 model_type: str = "bert",
                 model_name: str = "neuralmind/bert-base-portuguese-cased",
                 use_cuda: bool = True,
                 args: MultiLabelClassificationArgs = MultiLabelClassificationArgs()):

        self.model_type = model_type
        self.model_name = model_name
        self.use_cuda = use_cuda
        self.args = args
        self.model = None

    def get_params(self):
        return self.model.args.__dict__
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self.model.args, parameter, value)
        return self

    def fit(self,
            X: List[str],
            y: List[List[int]] | np.ndarray,
            metrics: Dict[str, callable] = {}, pos_weights = None) -> None:

        # define num_labels
        if isinstance(y, np.ndarray):
            num_labels = y.shape[1]
        else:
            num_labels = len(y[0])

        data = self._prepare_data(X, y)

        # Create a ClassificationModel
        self.model = MultiLabelClassificationModel(
            model_type=self.model_type,
            model_name=self.model_name,
            num_labels=num_labels,
            args=self.args,
            pos_weight=pos_weights,
            use_cuda=self.use_cuda)

        # Train the model
        self.model.train_model(
            train_df=data,
            eval_df=data,
            **metrics
        )

    def _predict(self, X: List[str] | np.ndarray) -> List[List[int]]:
        predictions, raw_outputs = self.model.predict(X)
        return predictions

    def predict(self, X: Union[str, List[str], np.ndarray]) -> List[List[int]]:
        if isinstance(X, str):
            return self._predict([X])
        elif isinstance(X, np.ndarray):
            return self._predict(X.tolist())
        else:
            return self._predict(X)

    def predict_proba(self, X: List[str]):
        predictions, raw_outputs = self.model.predict(X)
        return raw_outputs

    def score(self, X, y, metrics: Dict[str, callable] = {}) -> Dict[str, Any]:
        data = self._prepare_data(X, y)
        result, model_outputs, wrong_predictions = self.model.eval_model(
            eval_df=data,
            **metrics
        )
        return result

    def _prepare_data(self, X, y):
        data = []
        for x, _y in zip(X, y):
            data.append([x, _y])

        data = pd.DataFrame(data)
        data.columns = ["text", "labels"]
        return data

    def _delete_folders(
            folders: List[str] = ["cache_dir", "outputs", "runs"]):
        """Clean simpletransformers folders.

        Args:
        - folders: list of folders to clean
        """
        for folder in folders:
            if os.path.exists(folder):
                shutil.rmtree(folder, ignore_errors=True)

    def load_model(self):
        pass

In [12]:
def format_scores(scores: Dict[str, Dict[str, float]]) -> Dict[str, float | int]:
    """Format scores to be logged in mlflow.

    Args:
    - scores: classification report scores

    Returns:
    - formatted_scores: formatted scores
    """
    # Flatten the score dict
    scores = flatten(scores)

    # Remove whitespaces and "-" from keys
    scores = {k.replace(" ", "_").replace("-", "_"): v for k, v in scores.items()}

    # Remove "_support" items
    scores = {k: v for k, v in scores.items() if not k.endswith("_support")}

    return scores

## Hyperparameter tuning

Using the [Optuna](https://optuna.org/) library, we will perform a hyperparameter tuning to find the best hyperparameters for the model.

In [13]:
params["use_cuda"] = False

In [14]:
import optuna
from optuna.integration.mlflow import MLflowCallback


def objective(trial: optuna.Trial) -> float:
    with mlflow.start_run(nested=True):
        learning_rate = trial.suggest_float(
            "learning_rate", 1e-6, 1e-2)
        
        adam_epsilon = trial.suggest_float(
            "adam_epsilon", 1e-8, 1e-1)
        
        polynomial_decay_schedule_lr_end = trial.suggest_float(
            "polynomial_decay_schedule_lr_end", 1e-7, 1e-2)

        polynomial_decay_schedule_power = trial.suggest_float(
            "polynomial_decay_schedule_power", 0.1, 1.0)
        
        mlflow.log_params(trial.params)
        
        model = ToxicityTypeDetection(
            model_type=params["model_type"],
            model_name=params["model_name"],
            use_cuda=params["use_cuda"],
            args=MultiLabelClassificationArgs(
                num_train_epochs=params["num_train_epochs_per_child"],
                manual_seed=params["seed"],
                max_seq_length=params["max_seq_length"],
                learning_rate=learning_rate,
                adam_epsilon=adam_epsilon,
                polynomial_decay_schedule_lr_end=polynomial_decay_schedule_lr_end,
                polynomial_decay_schedule_power=polynomial_decay_schedule_power,
                overwrite_output_dir=True
            )
        )

        model.fit(
            X=X_train,
            y=y_train,
            pos_weights=pos_weights
        )

        # Evaluate the model
        result = model.score(X_test, y_test)

        scores = format_scores(
            classification_report(
                y_test, model.predict(X_test),
                target_names=toxicity_labels,
                output_dict=True, zero_division=0
            )
        )

        mlflow.log_metrics(result)
        mlflow.log_metrics(scores)

        return result["LRAP"]


study = optuna.create_study(direction="maximize")

study.optimize(
    objective,
    n_trials=20,
    n_jobs=-1,
    show_progress_bar=True
)

print("Number of finished trials: {}".format(len(study.trials)))

trial = study.best_trial

print(f"Best trial: {trial.number}")

# E.g. {'x': 2.002108042}
print(f"Best params: {study.best_params}")

[I 2022-11-04 14:40:30,475] A new study created in memory with name: no-name-8c7f4967-8402-46cd-a690-79d4f32790b5
c:\Python310\lib\site-packages\optuna\progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

2022-11-04 14:40:31,686 WARNING Connection pool is full, discarding connection: txshmzqv7b.us-east-1.awsapprunner.com. Connection pool size: 10
2022-11-04 14:40:31,717 WARNING Connection pool is full, discarding connection: txshmzqv7b.us-east-1.awsapprunner.com. Connection pool size: 10
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NO

  0%|          | 0/4272 [00:00<?, ?it/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClas

  0%|          | 0/4272 [00:01<?, ?it/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClas

## Training with best hyperparameters

In [ ]:
detector = ToxicityTypeDetection(
    model_type=params["model_type"],
    model_name=params["model_name"],
    use_cuda=params["use_cuda"],
    args=MultiLabelClassificationArgs(
        num_train_epochs=params["num_train_epochs"],
        manual_seed=params["seed"],
        max_seq_length=params["max_seq_length"],
        learning_rate=trial.params["learning_rate"],
        adam_epsilon=trial.params["adam_epsilon"],
        polynomial_decay_schedule_lr_end=trial.params["polynomial_decay_schedule_lr_end"],
        polynomial_decay_schedule_power=trial.params["polynomial_decay_schedule_power"],
        overwrite_output_dir=True
    )
)

detector.fit(
    X=X_train,
    y=y_train,
    pos_weights=pos_weights
)

# mlflow.log_params(params)
# mlflow.log_params(trial.params)

## Evaluation

In [ ]:
result = detector.score(X_test, y_test)
result

In [ ]:
y_pred = detector.predict(X_test)

scores = format_scores(
    classification_report(
        y_test, detector.predict(X_test),
        target_names=toxicity_labels,
        output_dict=True, zero_division=0
    )
)

# mlflow.log_metrics(result)
# mlflow.log_metrics(scores)

In [ ]:
print(
    classification_report(
        y_test, y_pred,
        target_names=toxicity_labels,
        digits=4, zero_division=0
    )
)

## Register best model

In [ ]:
# mlflow.pytorch.log_model(
#     pytorch_model=detector.model.model,
#     artifact_path="model",
#     conda_env="conda.yaml"
# )

In [ ]:
# mlflow.end_run()